In [4]:
# extract data with release filter
import os
import csv
# import pandas as pd
from pydriller import RepositoryMining
from radon.raw import analyze
from radon.metrics import h_visit
from radon.metrics import h_visit_ast
from radon.complexity import sorted_results
from pydriller.git_repository import GitRepository

fields = ['CommitID','filename','token_count','change_type','loc','lloc','sloc','comments',
          'multi','blank','code_comment','h1','h2','N1','N2','vocabulary','length',
          'calculated_length','volume', 'difficulty','effort','time','bugs']
# 'lines_added','lines_removed',
repo_updated = "/Users/hjain/Downloads/Sanika/Repo/pytest"
    
gr = GitRepository("/Users/hjain/Downloads/Sanika/Repo/pytest")
# no_commits = gitrepo.total_commits() #total number of commits

count = 0
# buggy_list = []

with open('final_dataset_01.csv', 'w') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames = fields)
    writer.writeheader()
    for commit in RepositoryMining(repo_updated,
                                   only_modifications_with_file_types=['.py'] ).traverse_commits():
        for modification in commit.modifications:
            filename = modification.filename
            hash_val = commit.hash
            change_type = modification.change_type
            token_count = modification.token_count 
#                 added = modification.added
#                 removed = modification.removed
            if filename.endswith(".py"):
                #Get bug inducing commits in a list
#                 commit1 = gr.get_commit(commit.hash)
#                 buggy_commits = gr.get_commits_last_modified_lines(commit1) #get bug inducing commits
#                 for x in buggy_commits:
#                     buggy_list.append(x)
                #Calculate metrics for all commits
                for r, d, f in os.walk(repo_updated):
                    for file in f:
                        if filename in file:
                            file_path = os.path.join(r, file)
                            with open(file_path) as f:
                                content = f.read()
                                file_analyze = analyze(content)
                                code_n_comment = file_analyze.loc+file_analyze.comments
                                file_ast = h_visit(content)
                                data = [{'CommitID':(hash_val), 
                                         'filename':(file_path),
                                     'token_count':(token_count),
                                     'change_type':(change_type),
#                                      'lines_added':(added),
#                                      'lines_removed':(removed),
                                     'loc':(file_analyze.loc),
                                     'lloc':(file_analyze.lloc),
                                     'sloc':(file_analyze.sloc),
                                     'comments':(file_analyze.comments),
                                     'multi':(file_analyze.multi),
                                     'blank':(file_analyze.blank),
                                     'code_comment':(code_n_comment),
                                     'h1':(file_ast.total.h1),
                                     'h2':(file_ast.total.h2),
                                     'N1':(file_ast.total.N1),
                                     'N2':(file_ast.total.N2),
                                     'vocabulary':(file_ast.total.vocabulary),
                                     'length':(file_ast.total.length),
                                     'calculated_length':(file_ast.total.calculated_length),
                                     'volume':(file_ast.total.volume),                     
                                     'difficulty':(file_ast.total.difficulty),
                                     'effort':(file_ast.total.effort),
                                     'time':(file_ast.total.time),
                                     'bugs':(file_ast.total.bugs),}]
                                writer.writerows(data)  
print("writing completed")
csvFile.close()

writing completed


In [9]:
repo_updated = "/Users/hjain/Downloads/Sanika/Repo/pytest"
gr = GitRepository("/Users/hjain/Downloads/Sanika/Repo/pytest")

buggy_list = [] = []

for commit in RepositoryMining(repo_updated,
                                   only_modifications_with_file_types=['.py'] ).traverse_commits():
    if "fix" in commit.msg:
        commit1 = gr.get_commit(commit.hash)
        buggy_commits = gr.get_commits_last_modified_lines(commit1)
        for x in buggy_commits:
            buggy_list.append(x)

In [10]:
len(buggy_list)

3225

In [7]:
import pandas as pd
df = pd.read_csv("final_dataset_01.csv")
df.loc[df['CommitID'].isin(buggy_list)]

,CommitID,filename,token_count,change_type,loc,lloc,sloc,comments,multi,blank,...,N1,N2,vocabulary,length,calculated_length,volume,difficulty,effort,time,bugs
0,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,2,2,2,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,d4c3b3ce85dcb054a49b8e7d207ea0e07eba149c,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,79.0,ModificationType.MODIFY,1455,414,1257,16,76,112,...,23,46,46,69,231.269273,381.125775,2.804878,1069.011320,59.389518,0.127042
3452,d4c3b3ce85dcb054a49b8e7d207ea0e07eba149c,/Users/hjain/Downloads/Sanika/Repo/pytest/src/...,79.0,ModificationType.MODIFY,683,369,490,24,64,111,...,63,122,118,185,747.312913,1273.288964,8.883495,11311.256332,628.403130,0.424430
3453,d4c3b3ce85dcb054a49b8e7d207ea0e07eba149c,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,262.0,ModificationType.MODIFY,1455,414,1257,16,76,112,...,23,46,46,69,231.269273,381.125775,2.804878,1069.011320,59.389518,0.127042
3454,8cbfaa1a4f81254983354c3d311055acc70b174b,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,1405.0,ModificationType.MODIFY,773,419,590,28,0,166,...,113,217,214,330,1598.394051,2554.684106,5.318627,13587.413012,754.856278,0.851561


In [8]:
df

,CommitID,filename,token_count,change_type,loc,lloc,sloc,comments,multi,blank,...,N1,N2,vocabulary,length,calculated_length,volume,difficulty,effort,time,bugs
0,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,2,2,2,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46429,5945c3fe8884dfd58f9b3d6172b5213b9c461f50,/Users/hjain/Downloads/Sanika/Repo/pytest/src/...,7285.0,ModificationType.MODIFY,1588,874,1147,90,127,228,...,163,307,297,470,2345.686113,3860.729987,9.319643,35980.624645,1998.923591,1.286910
46430,5945c3fe8884dfd58f9b3d6172b5213b9c461f50,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,6731.0,ModificationType.MODIFY,1952,704,1573,22,47,299,...,99,193,182,292,1319.072168,2192.276035,4.436782,9726.649994,540.369444,0.730759
46431,bc976dca3b6e0754c75ff6671f5ac16dff7281f4,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,6681.0,ModificationType.MODIFY,745,442,600,25,0,132,...,106,209,194,315,1409.438175,2393.972545,6.890110,16494.733911,916.374106,0.797991
46432,bc976dca3b6e0754c75ff6671f5ac16dff7281f4,/Users/hjain/Downloads/Sanika/Repo/pytest/src/...,6681.0,ModificationType.MODIFY,1526,852,942,44,247,292,...,90,172,165,262,1146.302085,1929.976420,7.973510,15388.686158,854.927009,0.643325


In [9]:
# Creating new dataframe 

df1 = pd.DataFrame(df, columns = ['CommitID','filename','token_count','change_type','loc','lloc','sloc','comments',
          'multi','blank','code_comment','h1','h2','N1','N2','vocabulary','length',
          'calculated_length','volume', 'difficulty','effort','time','bugs','defect']) 

# Generate result using pandas 
result = [] 
for value in df1["CommitID"]:
    if value in buggy_list:
        result.append(True)
    else:
        result.append(False)
    
df1["defect"] = result 
df1 

,CommitID,filename,token_count,change_type,loc,lloc,sloc,comments,multi,blank,...,N2,vocabulary,length,calculated_length,volume,difficulty,effort,time,bugs,defect
0,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True
1,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True
2,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,2,2,2,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True
3,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True
4,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,685.0,ModificationType.ADD,0,0,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46429,5945c3fe8884dfd58f9b3d6172b5213b9c461f50,/Users/hjain/Downloads/Sanika/Repo/pytest/src/...,7285.0,ModificationType.MODIFY,1588,874,1147,90,127,228,...,307,297,470,2345.686113,3860.729987,9.319643,35980.624645,1998.923591,1.286910,False
46430,5945c3fe8884dfd58f9b3d6172b5213b9c461f50,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,6731.0,ModificationType.MODIFY,1952,704,1573,22,47,299,...,193,182,292,1319.072168,2192.276035,4.436782,9726.649994,540.369444,0.730759,False
46431,bc976dca3b6e0754c75ff6671f5ac16dff7281f4,/Users/hjain/Downloads/Sanika/Repo/pytest/test...,6681.0,ModificationType.MODIFY,745,442,600,25,0,132,...,209,194,315,1409.438175,2393.972545,6.890110,16494.733911,916.374106,0.797991,False
46432,bc976dca3b6e0754c75ff6671f5ac16dff7281f4,/Users/hjain/Downloads/Sanika/Repo/pytest/src/...,6681.0,ModificationType.MODIFY,1526,852,942,44,247,292,...,172,165,262,1146.302085,1929.976420,7.973510,15388.686158,854.927009,0.643325,False


In [10]:
#writing release filter labelled data to CSV
df1.to_csv('final_dataset_01.csv')